<a href="https://colab.research.google.com/github/gregory-ch/shap_flex_porting/blob/main/demo_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"""
This module contains using user-defined trained models and prediction functions to compute approximate Shapley values for
single models. 
"""

import numpy as np
import pandas as pd
import random
import csv
import itertools

In [ ]:
class shapFlex_plus:
    def __init__(explain, reference = None, model, predict_function, target_features = None,
                     causal = None, causal_weights = None, sample_size = None, use_future = None)):
        self.explain = explain
        self.reference = reference if reference else explain
        self.model = model
        predict_function = predict_function
        self.target_features = target_features if target_features else explain.columns.tolist()
        self.causal = causal if causal else None
        self.causal_weights = causal_weights if causal_weights else None
        self.sample_size = sample_size if sample_size else 60
        self.use_future = use_future if target_features else False
        
        self.n_features = explain.shape[1]
        n_instances = self.reference.shape[0]
        
      def loop_over_monte_carlo_samples(i_size = self.samle_size, j_size = self.target_features):
        for i in range(i_size):
            for j in range(j_size):


1) собрать работающую функцию только с опциями по-умолчанию
2) из аргументов не по-умолчанию пока реализованы только: reference 
3) текущий раздел работы строки: 125-397, в строках инициализируется функция сэмплирования, проходит по двум петлям цикла. След задача написать цикл для i. 
